In [2]:
import os
import json
import numpy as np
import pandas as pd
import duckdb
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()

# -----------------------------
# INPUTS
# -----------------------------
TRIPS = PROJECT_ROOT / "data" / "interim" / "tripdata_2013_2025_clean.parquet"
CRASH_BIKE = PROJECT_ROOT / "data" / "processed" / "crashes_bike_liability.parquet"
WEATHER_HOURLY_DIR = PROJECT_ROOT / "data" / "raw" / "weather_hourly_openmeteo"

# -----------------------------
# OUTPUT DIR
# -----------------------------
OUT_DIR = PROJECT_ROOT / "data" / "processed" / "risk_hourly_mc"
OUT_DIR.mkdir(parents=True, exist_ok=True)

TMP_DIR = PROJECT_ROOT / "duckdb_tmp"
TMP_DIR.mkdir(parents=True, exist_ok=True)

# -----------------------------
# GRID + WINDOW
# -----------------------------
GRID_DEG = 0.025       
TMIN = "2020-01-01"
TMAX = "2026-01-01"

DB_PATH = OUT_DIR / f"analysis_bike_all_grid{int(GRID_DEG*10000):04d}.duckdb"
con = duckdb.connect(DB_PATH.as_posix())

# Make DuckDB stable on laptop RAM
con.execute("PRAGMA threads=4;")
con.execute("PRAGMA preserve_insertion_order=false;")
con.execute("PRAGMA memory_limit='8GB';")
con.execute(f"PRAGMA temp_directory='{TMP_DIR.as_posix()}';")

# Forcing rebuild prevents "same fit" due to cached files.
FORCE_REBUILD = True

# To make grid-model feasible: keep cells covering X% of total exposure (train window).
CELL_COVERAGE = 1.0    # 0.95..0.995 (higher => more cells => slower)

print("DuckDB:", DB_PATH)
print("GRID_DEG:", GRID_DEG, "| FORCE_REBUILD:", FORCE_REBUILD, "| CELL_COVERAGE:", CELL_COVERAGE)
print("Window:", TMIN, "→", TMAX)


DuckDB: /Users/patricknussbaum/Desktop/projects/city-bike/data/processed/risk_hourly_mc/analysis_bike_all_grid0250.duckdb
GRID_DEG: 0.025 | FORCE_REBUILD: True | CELL_COVERAGE: 1.0
Window: 2020-01-01 → 2026-01-01


In [3]:
crash_cell_hour_path = OUT_DIR / "crash_cell_hour.parquet"

if FORCE_REBUILD and crash_cell_hour_path.exists():
    crash_cell_hour_path.unlink()
    print("Deleted:", crash_cell_hour_path)

if crash_cell_hour_path.exists():
    print("Exists, skipping:", crash_cell_hour_path)
else:
    con.execute(f"""
    COPY (
      WITH base AS (
        SELECT
          -- Preferred: DATE + TIME (works if crash_date is DATE/TIMESTAMP and crash_time is TIME-like)
          (try_cast(crash_date AS DATE) + try_cast(crash_time AS TIME))::TIMESTAMP AS crash_ts_dt,

          -- Fallback: parse crash_time as "H:MM" / "HH:MM" strings
          try_strptime(
            strftime(try_cast(crash_date AS DATE), '%Y-%m-%d') || ' ' ||
            printf(
              '%02d:%02d',
              try_cast(regexp_extract(CAST(crash_time AS VARCHAR), '^(\\d{{1,2}})', 1) AS INTEGER),
              try_cast(regexp_extract(CAST(crash_time AS VARCHAR), ':(\\d{{2}})', 1) AS INTEGER)
            ),
            '%Y-%m-%d %H:%M'
          ) AS crash_ts_str,

          latitude,
          longitude
        FROM read_parquet('{CRASH_BIKE.as_posix()}')
      ),
      ts AS (
        SELECT
          COALESCE(crash_ts_dt, crash_ts_str) AS crash_ts,
          latitude,
          longitude
        FROM base
      ),
      binned AS (
        SELECT
          date_trunc('hour', crash_ts) AS hour_ts,
          floor(latitude  / {GRID_DEG}) * {GRID_DEG} AS grid_lat,
          floor(longitude / {GRID_DEG}) * {GRID_DEG} AS grid_lng,
          1 AS y_bike
        FROM ts
        WHERE crash_ts IS NOT NULL
          AND latitude IS NOT NULL AND longitude IS NOT NULL
          AND crash_ts >= TIMESTAMP '{TMIN}'
          AND crash_ts <  TIMESTAMP '{TMAX}'
      )
      SELECT
        hour_ts,
        CAST(grid_lat AS DOUBLE) AS grid_lat,
        CAST(grid_lng AS DOUBLE) AS grid_lng,
        CAST(grid_lat AS VARCHAR) || '_' || CAST(grid_lng AS VARCHAR) AS cell_id,
        SUM(y_bike) AS y_bike
      FROM binned
      GROUP BY 1,2,3,4
    )
    TO '{crash_cell_hour_path.as_posix()}'
    (FORMAT PARQUET);
    """)

    print("Wrote:", crash_cell_hour_path)

# sanity
print(con.execute(f"""
SELECT COUNT(*) n, MIN(hour_ts) min_ts, MAX(hour_ts) max_ts
FROM read_parquet('{crash_cell_hour_path.as_posix()}');
""").fetch_df())


Deleted: /Users/patricknussbaum/Desktop/projects/city-bike/data/processed/risk_hourly_mc/crash_cell_hour.parquet
Wrote: /Users/patricknussbaum/Desktop/projects/city-bike/data/processed/risk_hourly_mc/crash_cell_hour.parquet
       n              min_ts              max_ts
0  43871 2020-01-01 10:00:00 2025-12-31 20:00:00


In [ ]:
exposure_cell_hour_path = OUT_DIR / "exposure_cell_hour.parquet"

if FORCE_REBUILD and exposure_cell_hour_path.exists():
    exposure_cell_hour_path.unlink()
    print("Deleted:", exposure_cell_hour_path)

if exposure_cell_hour_path.exists():
    print("Exists, skipping:", exposure_cell_hour_path)
else:
    # Process year-by-year to avoid memory issues
    print(f"Creating exposure (year-by-year) for {TMIN} to {TMAX}...")
    
    # Extract year range from TMIN/TMAX
    import datetime
    year_min = datetime.datetime.fromisoformat(TMIN).year
    year_max = datetime.datetime.fromisoformat(TMAX).year
    years = range(year_min, year_max + 1)
    
    temp_files = []
    
    for year in years:
        year_start = max(f"{year}-01-01", TMIN)  # Don't go before TMIN
        if year == year_max:
            year_end = TMAX  # Use TMAX for last year
        else:
            year_end = f"{year+1}-01-01"
        
        temp_path = OUT_DIR / f"_temp_exp_{year}.parquet"
        temp_files.append(temp_path)
        
        if temp_path.exists():
            print(f"  Year {year}: already exists, skipping")
            continue
            
        print(f"  Processing year {year} ({year_start} to {year_end})...")
        con.execute(f"""
        COPY (
          WITH trips AS (
            SELECT
              try_cast(started_at AS TIMESTAMP) AS started_at,
              try_cast(ended_at   AS TIMESTAMP) AS ended_at,
              start_lat, start_lng,
              duration_sec
            FROM read_parquet('{TRIPS.as_posix()}')
            WHERE try_cast(started_at AS TIMESTAMP) >= TIMESTAMP '{year_start}'
              AND try_cast(started_at AS TIMESTAMP) <  TIMESTAMP '{year_end}'
              AND start_lat IS NOT NULL AND start_lng IS NOT NULL
              AND ended_at IS NOT NULL
              AND duration_sec IS NOT NULL
              AND duration_sec > 0
              AND duration_sec < 4*60*60
          ),
          binned AS (
            SELECT
              floor(start_lat / {GRID_DEG}) * {GRID_DEG} AS grid_lat,
              floor(start_lng / {GRID_DEG}) * {GRID_DEG} AS grid_lng,
              CAST(floor(start_lat / {GRID_DEG}) * {GRID_DEG} AS VARCHAR) || '_' ||
              CAST(floor(start_lng / {GRID_DEG}) * {GRID_DEG} AS VARCHAR) AS cell_id,
              started_at,
              ended_at
            FROM trips
          ),
          expanded AS (
            SELECT
              cell_id, grid_lat, grid_lng,
              gs AS hour_ts,
              started_at, ended_at
            FROM binned
            CROSS JOIN generate_series(
              date_trunc('hour', started_at),
              date_trunc('hour', ended_at),
              INTERVAL '1 hour'
            ) AS t(gs)
          ),
          overlap AS (
            SELECT
              cell_id, grid_lat, grid_lng, hour_ts,
              GREATEST(
                0,
                EXTRACT(EPOCH FROM LEAST(ended_at, hour_ts + INTERVAL '1 hour')
                              - GREATEST(started_at, hour_ts))
              ) AS overlap_sec
            FROM expanded
          )
          SELECT
            hour_ts,
            CAST(grid_lat AS DOUBLE) AS grid_lat,
            CAST(grid_lng AS DOUBLE) AS grid_lng,
            cell_id,
            SUM(overlap_sec)/60.0 AS exposure_min
          FROM overlap
          WHERE overlap_sec > 0
          GROUP BY 1,2,3,4
        )
        TO '{temp_path.as_posix()}'
        (FORMAT PARQUET);
        """)
    
    # Combine all years
    print("  Combining years...")
    all_files = "', '".join([f.as_posix() for f in temp_files if f.exists()])
    con.execute(f"""
    COPY (
      SELECT * FROM read_parquet(['{all_files}'])
    )
    TO '{exposure_cell_hour_path.as_posix()}'
    (FORMAT PARQUET);
    """)
    
    # Cleanup
    for f in temp_files:
        if f.exists():
            f.unlink()
    
    print("Wrote:", exposure_cell_hour_path)

# sanity
print(con.execute(f"""
SELECT COUNT(*) n, MIN(hour_ts) min_ts, MAX(hour_ts) max_ts, SUM(exposure_min) total_exp_min
FROM read_parquet('{exposure_cell_hour_path.as_posix()}');
""").fetch_df())

Deleted: /Users/patricknussbaum/Desktop/projects/city-bike/data/processed/risk_hourly_mc/exposure_cell_hour.parquet
Creating exposure (year-by-year) for 2020-01-01 to 2026-01-01...
  Processing year 2020 (2020-01-01 to 2021-01-01)...
  Processing year 2021 (2021-01-01 to 2022-01-01)...
  Processing year 2022 (2022-01-01 to 2023-01-01)...
  Processing year 2023 (2023-01-01 to 2024-01-01)...


In [4]:
crashes_citywide_path  = OUT_DIR / "crashes_bike_hour_citywide.parquet"
exposure_citywide_path = OUT_DIR / "exposure_hour_citywide.parquet"

if FORCE_REBUILD:
    if crashes_citywide_path.exists(): crashes_citywide_path.unlink()
    if exposure_citywide_path.exists(): exposure_citywide_path.unlink()

con.execute(f"""
COPY (
  SELECT hour_ts, SUM(y_bike) AS y_bike
  FROM read_parquet('{crash_cell_hour_path.as_posix()}')
  GROUP BY 1
  ORDER BY 1
)
TO '{crashes_citywide_path.as_posix()}'
(FORMAT PARQUET);
""")

con.execute(f"""
COPY (
  SELECT hour_ts, SUM(exposure_min) AS exposure_min
  FROM read_parquet('{exposure_cell_hour_path.as_posix()}')
  GROUP BY 1
  ORDER BY 1
)
TO '{exposure_citywide_path.as_posix()}'
(FORMAT PARQUET);
""")

print("Wrote:", crashes_citywide_path)
print("Wrote:", exposure_citywide_path)
print(con.execute(f"SELECT COUNT(*) FROM read_parquet('{crashes_citywide_path.as_posix()}')").fetchone()[0],
      "hours crashes citywide")
print(con.execute(f"SELECT COUNT(*) FROM read_parquet('{exposure_citywide_path.as_posix()}')").fetchone()[0],
      "hours exposure citywide")


Wrote: /Users/patricknussbaum/Desktop/projects/city-bike/data/processed/risk_hourly_mc/crashes_bike_hour_citywide.parquet
Wrote: /Users/patricknussbaum/Desktop/projects/city-bike/data/processed/risk_hourly_mc/exposure_hour_citywide.parquet
26015 hours crashes citywide
51763 hours exposure citywide


In [5]:
con.execute(f"""
CREATE OR REPLACE VIEW weather_hourly AS
SELECT
  CAST(timestamp AS TIMESTAMP) AS hour_ts,
  temp, prcp, snow, wspd
FROM read_parquet('{(WEATHER_HOURLY_DIR.as_posix() + "/**/*.parquet")}')
WHERE hour_ts >= TIMESTAMP '{TMIN}'
  AND hour_ts <  TIMESTAMP '{TMAX}';
""")

print("weather_hourly rows:", con.execute("SELECT COUNT(*) FROM weather_hourly").fetchone()[0])
print("weather range:", con.execute("SELECT MIN(hour_ts), MAX(hour_ts) FROM weather_hourly").fetchone())


weather_hourly rows: 52561
weather range: (datetime.datetime(2020, 1, 1, 0, 0), datetime.datetime(2025, 12, 30, 0, 0))


In [6]:
city_train_path = OUT_DIR / "city_train_bike_all_2020_2024.parquet"
city_2025_path  = OUT_DIR / "city_2025_bike_all.parquet"

if FORCE_REBUILD:
    if city_train_path.exists(): city_train_path.unlink()
    if city_2025_path.exists(): city_2025_path.unlink()

con.execute(f"""
COPY (
  WITH e AS (
    SELECT hour_ts, exposure_min
    FROM read_parquet('{exposure_citywide_path.as_posix()}')
    WHERE hour_ts >= TIMESTAMP '2020-01-01' AND hour_ts < TIMESTAMP '2025-01-01'
  ),
  c AS (
    SELECT hour_ts, y_bike
    FROM read_parquet('{crashes_citywide_path.as_posix()}')
    WHERE hour_ts >= TIMESTAMP '2020-01-01' AND hour_ts < TIMESTAMP '2025-01-01'
  )
  SELECT
    e.hour_ts,
    e.exposure_min,
    COALESCE(c.y_bike, 0) AS y_bike,
    EXTRACT(HOUR  FROM e.hour_ts) AS hour_of_day,
    EXTRACT(DOW   FROM e.hour_ts) AS dow,
    EXTRACT(MONTH FROM e.hour_ts) AS month,
    w.temp, w.prcp, w.snow, w.wspd
  FROM e
  LEFT JOIN c USING(hour_ts)
  LEFT JOIN weather_hourly w USING(hour_ts)
  WHERE e.exposure_min > 0
  ORDER BY e.hour_ts
)
TO '{city_train_path.as_posix()}'
(FORMAT PARQUET);
""")

con.execute(f"""
COPY (
  WITH e AS (
    SELECT hour_ts, exposure_min
    FROM read_parquet('{exposure_citywide_path.as_posix()}')
    WHERE hour_ts >= TIMESTAMP '2025-01-01' AND hour_ts < TIMESTAMP '2026-01-01'
  )
  SELECT
    e.hour_ts,
    e.exposure_min,
    EXTRACT(HOUR  FROM e.hour_ts) AS hour_of_day,
    EXTRACT(DOW   FROM e.hour_ts) AS dow,
    EXTRACT(MONTH FROM e.hour_ts) AS month,
    w.temp, w.prcp, w.snow, w.wspd
  FROM e
  LEFT JOIN weather_hourly w USING(hour_ts)
  WHERE e.exposure_min > 0
  ORDER BY e.hour_ts
)
TO '{city_2025_path.as_posix()}'
(FORMAT PARQUET);
""")

print("Wrote:", city_train_path, "rows=", con.execute(f"SELECT COUNT(*) FROM read_parquet('{city_train_path.as_posix()}')").fetchone()[0])
print("Wrote:", city_2025_path,  "rows=", con.execute(f"SELECT COUNT(*) FROM read_parquet('{city_2025_path.as_posix()}')").fetchone()[0])


Wrote: /Users/patricknussbaum/Desktop/projects/city-bike/data/processed/risk_hourly_mc/city_train_bike_all_2020_2024.parquet rows= 43752
Wrote: /Users/patricknussbaum/Desktop/projects/city-bike/data/processed/risk_hourly_mc/city_2025_bike_all.parquet rows= 8017


In [7]:
grid_train_path = OUT_DIR / "grid_train_cell_hour_2020_2024.parquet"
cells_keep_path = OUT_DIR / "grid_cells_keep_2020_2024.parquet"

if FORCE_REBUILD:
    if grid_train_path.exists(): grid_train_path.unlink()
    if cells_keep_path.exists(): cells_keep_path.unlink()

# 1) Determine top exposure cells covering CELL_COVERAGE of total exposure
con.execute(f"""
COPY (
  WITH cell_exp AS (
    SELECT
      cell_id,
      SUM(exposure_min) AS exp_sum
    FROM read_parquet('{exposure_cell_hour_path.as_posix()}')
    WHERE hour_ts >= TIMESTAMP '2020-01-01'
      AND hour_ts <  TIMESTAMP '2025-01-01'
    GROUP BY 1
  ),
  ranked AS (
    SELECT
      cell_id,
      exp_sum,
      SUM(exp_sum) OVER (ORDER BY exp_sum DESC) AS cum_exp,
      SUM(exp_sum) OVER () AS total_exp
    FROM cell_exp
  )
  SELECT
    cell_id,
    exp_sum,
    cum_exp / NULLIF(total_exp,0) AS cum_share
  FROM ranked
  WHERE cum_exp / NULLIF(total_exp,0) <= {CELL_COVERAGE}
)
TO '{cells_keep_path.as_posix()}'
(FORMAT PARQUET);
""")

print("Wrote:", cells_keep_path)

# 2) Build grid training mart: one row per (cell_id, hour_ts)
con.execute(f"""
COPY (
  WITH e AS (
    SELECT
      hour_ts, cell_id, grid_lat, grid_lng, exposure_min
    FROM read_parquet('{exposure_cell_hour_path.as_posix()}')
    WHERE hour_ts >= TIMESTAMP '2020-01-01'
      AND hour_ts <  TIMESTAMP '2025-01-01'
      AND exposure_min > 0
  ),
  e_keep AS (
    SELECT e.*
    FROM e
    INNER JOIN read_parquet('{cells_keep_path.as_posix()}') k USING(cell_id)
  ),
  c AS (
    SELECT hour_ts, cell_id, y_bike
    FROM read_parquet('{crash_cell_hour_path.as_posix()}')
    WHERE hour_ts >= TIMESTAMP '2020-01-01'
      AND hour_ts <  TIMESTAMP '2025-01-01'
  )
  SELECT
    e_keep.hour_ts,
    e_keep.cell_id,
    e_keep.grid_lat,
    e_keep.grid_lng,
    e_keep.exposure_min,
    COALESCE(c.y_bike, 0) AS y_bike,
    EXTRACT(HOUR  FROM e_keep.hour_ts) AS hour_of_day,
    EXTRACT(DOW   FROM e_keep.hour_ts) AS dow,
    EXTRACT(MONTH FROM e_keep.hour_ts) AS month,
    w.temp, w.prcp, w.snow, w.wspd
  FROM e_keep
  LEFT JOIN c USING(hour_ts, cell_id)
  LEFT JOIN weather_hourly w USING(hour_ts)
)
TO '{grid_train_path.as_posix()}'
(FORMAT PARQUET);
""")

print("Wrote:", grid_train_path)
print(con.execute(f"""
SELECT COUNT(*) n, AVG(y_bike) y_mean, AVG(exposure_min) exp_mean
FROM read_parquet('{grid_train_path.as_posix()}');
""").fetch_df())


Wrote: /Users/patricknussbaum/Desktop/projects/city-bike/data/processed/risk_hourly_mc/grid_cells_keep_2020_2024.parquet
Wrote: /Users/patricknussbaum/Desktop/projects/city-bike/data/processed/risk_hourly_mc/grid_train_cell_hour_2020_2024.parquet
         n    y_mean     exp_mean
0  1818038  0.014245  1211.783519


In [16]:

import statsmodels.api as sm
import statsmodels.formula.api as smf

TARGET = "y_bike"
MIN_EXPOSURE_MIN = 50.0
EPS = 1e-6
weather_cols = ["temp","prcp","snow","wspd"]

# Load (grid train) to pandas
train_df = con.execute(f"SELECT * FROM read_parquet('{grid_train_path.as_posix()}')").fetch_df()

# clean
need = ["exposure_min", TARGET, "hour_of_day","dow","month","grid_lat","grid_lng"] + weather_cols
train_df = train_df.dropna(subset=need).copy()
train_df = train_df[train_df["exposure_min"] >= MIN_EXPOSURE_MIN].copy()

# offset
train_df["log_exposure"] = np.log(train_df["exposure_min"].values + EPS)

# categories
train_df["hour_of_day"] = train_df["hour_of_day"].astype(int).astype("category")
train_df["dow"] = train_df["dow"].astype(int).astype("category")
train_df["month"] = train_df["month"].astype(int).astype("category")

weather_stats = {}
for col in weather_cols:
    m = float(train_df[col].mean())
    s = float(train_df[col].std())
    if s == 0 or np.isnan(s):
        train_df.drop(columns=[col], inplace=True)
    else:
        weather_stats[col] = (m, s)
        train_df[col] = (train_df[col] - m) / s

active_weather_cols = [c for c in weather_cols if c in train_df.columns]

# WICHTIG: Normalize spatial features too!
lat_mean, lat_std = train_df["grid_lat"].mean(), train_df["grid_lat"].std()
lng_mean, lng_std = train_df["grid_lng"].mean(), train_df["grid_lng"].std()

train_df["grid_lat_norm"] = (train_df["grid_lat"] - lat_mean) / lat_std
train_df["grid_lng_norm"] = (train_df["grid_lng"] - lng_mean) / lng_std

# spatial features (normalized versions)
train_df["lat2"] = train_df["grid_lat_norm"]**2
train_df["lng2"] = train_df["grid_lng_norm"]**2
train_df["lat_lng"] = train_df["grid_lat_norm"] * train_df["grid_lng_norm"]

# model formula (GRID MODEL) - use normalized versions
rhs = [
    "C(hour_of_day)", "C(dow)", "C(month)",
    "grid_lat_norm", "grid_lng_norm", "lat2", "lng2", "lat_lng"
] + active_weather_cols

formula = f"{TARGET} ~ " + " + ".join(rhs)
print("Formula:", formula)

# ---- Poisson GLM ----
poisson_model = smf.glm(
    formula=formula,
    data=train_df,
    family=sm.families.Poisson(),
    offset=train_df["log_exposure"]
)
poisson_res = poisson_model.fit(cov_type="HC0")
print("\n=== POISSON (grid model) ===")
print(poisson_res.summary())

pearson_chi2 = float(np.sum(poisson_res.resid_pearson**2))
disp_poiss = pearson_chi2 / float(poisson_res.df_resid)
print("\nPoisson overdispersion χ²/df:", disp_poiss)

# ---- Negative Binomial GLM (fixed alpha heuristic) ----
mu_bar = float(poisson_res.mu.mean())
alpha_init = max((disp_poiss - 1.0) / max(mu_bar, 1e-9), 1e-8)

nb_model = smf.glm(
    formula=formula,
    data=train_df,
    family=sm.families.NegativeBinomial(alpha=alpha_init),
    offset=train_df["log_exposure"]
)
nb_res = nb_model.fit(cov_type="HC0")

print("\n=== NEG BIN (grid model) ===")
print(nb_res.summary())
print("\nNB alpha used:", float(nb_res.model.family.alpha))

meta = {
    "GRID_DEG": GRID_DEG,
    "CELL_COVERAGE": CELL_COVERAGE,
    "weather_stats": {k: {"mean": v[0], "std": v[1]} for k,v in weather_stats.items()},
    "spatial_stats": {
        "lat_mean": float(lat_mean),
        "lat_std": float(lat_std),
        "lng_mean": float(lng_mean),
        "lng_std": float(lng_std)
    },
    "formula": formula,
    "active_weather_cols": active_weather_cols
}
meta_path = OUT_DIR / "model_meta_bike_all.json"
meta_path.write_text(json.dumps(meta, indent=2))
print("\nSaved:", meta_path)

Formula: y_bike ~ C(hour_of_day) + C(dow) + C(month) + grid_lat_norm + grid_lng_norm + lat2 + lng2 + lat_lng + temp + prcp + snow + wspd

=== POISSON (grid model) ===
                 Generalized Linear Model Regression Results                  
Dep. Variable:                 y_bike   No. Observations:              1477060
Model:                            GLM   Df Residuals:                  1477010
Model Family:                 Poisson   Df Model:                           49
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.2025e+05
Date:                Tue, 30 Dec 2025   Deviance:                   1.9218e+05
Time:                        17:10:10   Pearson chi2:                 2.14e+06
No. Iterations:                     8   Pseudo R-squ. (CS):            0.01077
Covariance Type:                  HC0                                         
                           coef    std err 

In [ ]:
# ADD THIS RIGHT BEFORE CREATING SPATIAL FEATURES:
train_df = con.execute(f"SELECT * FROM read_parquet('{grid_train_path.as_posix()}')").fetch_df()

print("="*70)
print("SPATIAL FEATURES DEBUG")
print("="*70)

print(f"\nGrid configuration:")
print(f"  GRID_DEG: {GRID_DEG}")
print(f"  Unique cells: {train_df['cell_id'].nunique() if 'cell_id' in train_df.columns else 'N/A'}")

print(f"\nSpatial raw data:")
print(f"  grid_lat range: [{train_df['grid_lat'].min():.6f}, {train_df['grid_lat'].max():.6f}]")
print(f"  grid_lng range: [{train_df['grid_lng'].min():.6f}, {train_df['grid_lng'].max():.6f}]")
print(f"  grid_lat spread: {train_df['grid_lat'].max() - train_df['grid_lat'].min():.6f}")
print(f"  grid_lng spread: {train_df['grid_lng'].max() - train_df['grid_lng'].min():.6f}")

lat_mean = float(train_df["grid_lat"].mean())
lat_std = float(train_df["grid_lat"].std())
lng_mean = float(train_df["grid_lng"].mean())
lng_std = float(train_df["grid_lng"].std())

print(f"\nSpatial statistics:")
print(f"  lat_mean: {lat_mean:.6f}")
print(f"  lat_std:  {lat_std:.6f}  ← CHECK THIS!")
print(f"  lng_mean: {lng_mean:.6f}")
print(f"  lng_std:  {lng_std:.6f}  ← CHECK THIS!")

# ============================================================================
# CRITICAL: Check for near-zero std!
# ============================================================================
if lat_std < 0.001 or lng_std < 0.001:
    print(f"\n WARNING: STD TOO SMALL!")
    print(f"   This will cause EXTREME values after normalization!")
    print(f"   Division by {lat_std:.6f} or {lng_std:.6f} → Inf/NaN!")
    
# Try normalizing and check results
train_df["grid_lat_norm"] = (train_df["grid_lat"] - lat_mean) / lat_std
train_df["grid_lng_norm"] = (train_df["grid_lng"] - lng_mean) / lng_std

print(f"\nNormalized values:")
print(f"  grid_lat_norm range: [{train_df['grid_lat_norm'].min():.2f}, {train_df['grid_lat_norm'].max():.2f}]")
print(f"  grid_lng_norm range: [{train_df['grid_lng_norm'].min():.2f}, {train_df['grid_lng_norm'].max():.2f}]")

# Check for bad values
n_inf_lat = np.isinf(train_df['grid_lat_norm']).sum()
n_nan_lat = train_df['grid_lat_norm'].isna().sum()
n_inf_lng = np.isinf(train_df['grid_lng_norm']).sum()
n_nan_lng = train_df['grid_lng_norm'].isna().sum()

print(f"\nBad values check:")
print(f"  grid_lat_norm: inf={n_inf_lat}, nan={n_nan_lat}")
print(f"  grid_lng_norm: inf={n_inf_lng}, nan={n_nan_lng}")

if n_inf_lat > 0 or n_nan_lat > 0 or n_inf_lng > 0 or n_nan_lng > 0:
    print(f"\nBAD VALUES FOUND! THIS WILL CRASH!")

# Create polynomial features
train_df["lat2"] = train_df["grid_lat_norm"]**2
train_df["lng2"] = train_df["grid_lng_norm"]**2
train_df["lat_lng"] = train_df["grid_lat_norm"] * train_df["grid_lng_norm"]

print(f"\nPolynomial features:")
print(f"  lat2 range: [{train_df['lat2'].min():.2f}, {train_df['lat2'].max():.2f}]")
print(f"  lng2 range: [{train_df['lng2'].min():.2f}, {train_df['lng2'].max():.2f}]")
print(f"  lat_lng range: [{train_df['lat_lng'].min():.2f}, {train_df['lat_lng'].max():.2f}]")

# Check polynomials for bad values
for col in ["lat2", "lng2", "lat_lng"]:
    n_inf = np.isinf(train_df[col]).sum()
    n_nan = train_df[col].isna().sum()
    print(f"  {col}: inf={n_inf}, nan={n_nan}")
    if n_inf > 0 or n_nan > 0:
        print(f"  BAD VALUES IN {col}!")

print("="*70)

SPATIAL FEATURES DEBUG

Grid configuration:
  GRID_DEG: 0.025
  Unique cells: 64

Spatial raw data:
  grid_lat range: [40.600000, 40.875000]
  grid_lng range: [-74.100000, -73.850000]
  grid_lat spread: 0.275000
  grid_lng spread: 0.250000

Spatial statistics:
  lat_mean: 40.734289
  lat_std:  0.064653  ← CHECK THIS!
  lng_mean: -73.962372
  lng_std:  0.041358  ← CHECK THIS!

Normalized values:
  grid_lat_norm range: [-2.08, 2.18]
  grid_lng_norm range: [-3.33, 2.72]

Bad values check:
  grid_lat_norm: inf=0, nan=0
  grid_lng_norm: inf=0, nan=0

Polynomial features:
  lat2 range: [0.02, 4.74]
  lng2 range: [0.09, 11.07]
  lat_lng range: [-1.38, 3.58]
  lat2: inf=0, nan=0
  lng2: inf=0, nan=0
  lat_lng: inf=0, nan=0


In [18]:
grid_2025_path = OUT_DIR / "grid_2025_cell_hour_bike_all.parquet"

if FORCE_REBUILD and grid_2025_path.exists():
    grid_2025_path.unlink()

con.execute(f"""
COPY (
  WITH e AS (
    SELECT
      hour_ts, cell_id, grid_lat, grid_lng, exposure_min
    FROM read_parquet('{exposure_cell_hour_path.as_posix()}')
    WHERE hour_ts >= TIMESTAMP '2025-01-01'
      AND hour_ts <  TIMESTAMP '2026-01-01'
      AND exposure_min > 0
  ),
  e_keep AS (
    SELECT e.*
    FROM e
    INNER JOIN read_parquet('{cells_keep_path.as_posix()}') k USING(cell_id)
  )
  SELECT
    e_keep.hour_ts,
    e_keep.cell_id,
    e_keep.grid_lat,
    e_keep.grid_lng,
    e_keep.exposure_min,
    EXTRACT(HOUR  FROM e_keep.hour_ts) AS hour_of_day,
    EXTRACT(DOW   FROM e_keep.hour_ts) AS dow,
    EXTRACT(MONTH FROM e_keep.hour_ts) AS month,
    w.temp, w.prcp, w.snow, w.wspd
  FROM e_keep
  LEFT JOIN weather_hourly w USING(hour_ts)
  ORDER BY hour_ts, cell_id
)
TO '{grid_2025_path.as_posix()}'
(FORMAT PARQUET);
""")

print("Wrote:", grid_2025_path)
print(con.execute(f"SELECT COUNT(*) n FROM read_parquet('{grid_2025_path.as_posix()}')").fetch_df())


Wrote: /Users/patricknussbaum/Desktop/projects/city-bike/data/processed/risk_hourly_mc/grid_2025_cell_hour_bike_all.parquet
        n
0  412528


In [ ]:
import patsy
from numpy.random import default_rng

rng = default_rng(42)
S = 500  # 5 years × 100 iterations each

# ============================================================================
# STEP 1: Load Weather for ALL Years (by day-of-year + hour)
# ============================================================================
print("="*70)
print("PREPARING YEAR-LEVEL WEATHER BOOTSTRAP (day-of-year mapping)")
print("="*70)

# Load training weather
train_weather = con.execute(f"SELECT * FROM read_parquet('{grid_train_path.as_posix()}')").fetch_df()
train_weather["hour_ts"] = pd.to_datetime(train_weather["hour_ts"])
train_weather["year"] = train_weather["hour_ts"].dt.year
train_weather["dayofyear"] = train_weather["hour_ts"].dt.dayofyear
train_weather["hour"] = train_weather["hour_ts"].dt.hour

weather_cols = ["temp", "prcp", "snow", "wspd"]
available_years = [2020, 2021, 2022, 2023, 2024]

# Build lookup: year → (dayofyear, hour) → weather
year_weather_dict = {}

for year in available_years:
    year_data = train_weather[train_weather['year'] == year].copy()
    
    # Aggregate to (dayofyear, hour) - take median if multiple observations
    year_agg = year_data.groupby(['dayofyear', 'hour'])[weather_cols].median().reset_index()
    
    # Convert to dict
    year_dict = {}
    for _, row in year_agg.iterrows():
        key = (int(row['dayofyear']), int(row['hour']))
        year_dict[key] = {col: row[col] for col in weather_cols}
    
    year_weather_dict[year] = year_dict
    print(f"  Year {year}: {len(year_dict)} unique (dayofyear, hour) combinations")

# Check coverage
test_key = (15, 14)  # 15th day of year, 14:00
print(f"\nDay 15, hour 14 across years:")
for year in available_years:
    if test_key in year_weather_dict[year]:
        w = year_weather_dict[year][test_key]
        print(f"  {year}: temp={w['temp']:+.1f}°C, prcp={w['prcp']:.1f}mm")

# ============================================================================
# STEP 2: Prepare 2025 Data
# ============================================================================
df2025 = con.execute(f"SELECT * FROM read_parquet('{grid_2025_path.as_posix()}')").fetch_df()
df2025 = df2025.dropna(subset=["exposure_min","hour_of_day","dow","month","grid_lat","grid_lng"]).copy()
df2025 = df2025[df2025["exposure_min"] > 0].copy()

df2025["hour_ts"] = pd.to_datetime(df2025["hour_ts"])
df2025["dayofyear"] = df2025["hour_ts"].dt.dayofyear.astype(int)
df2025["hour"] = df2025["hour_ts"].dt.hour.astype(int)

df2025["hour_of_day"] = df2025["hour_of_day"].astype(int).astype("category")
df2025["dow"] = df2025["dow"].astype(int).astype("category")
df2025["month"] = df2025["month"].astype(int).astype("category")

if meta_path.exists():
    meta = json.loads(meta_path.read_text())
    wstats = meta["weather_stats"]
    sstats = meta["spatial_stats"]
else:
    raise ValueError("meta_path not found!")

print(f"\nPrepared 2025 data: {len(df2025)} rows")

# Pre-extract keys
df2025_keys = list(zip(df2025['dayofyear'], df2025['hour']))

# ============================================================================
# STEP 3: Simulation with Day-of-Year Weather Mapping
# ============================================================================
def simulate_totals_year_weather(res, use_nb: bool):
    """
    For each simulation: Pick ONE year and map 2025 dates to that year's weather
    """
    design_info = res.model.data.design_info
    alpha_nb = float(res.model.family.alpha) if use_nb else 0.0
    
    totals = np.zeros(S, dtype=np.int64)
    
    for s in range(S):
        if s % 50 == 0:
            print(f"  Simulation {s}/{S}...")
        
        # ----------------------------------------------------------------
        # A) Pick ONE year's weather
        # ----------------------------------------------------------------
        sampled_year = rng.choice(available_years)
        year_weather = year_weather_dict[sampled_year]
        
        # ----------------------------------------------------------------
        # B) Map 2025 (dayofyear, hour) → sampled year weather
        # ----------------------------------------------------------------
        weather_matrix = np.zeros((len(df2025), 4))
        
        for i, key in enumerate(df2025_keys):
            if key in year_weather:
                w = year_weather[key]
                weather_matrix[i] = [w['temp'], w['prcp'], w['snow'], w['wspd']]
            # else: keep 0 (missing weather)
        
        # Apply to df_sim
        df_sim = df2025.copy()
        df_sim['temp'] = weather_matrix[:, 0]
        df_sim['prcp'] = weather_matrix[:, 1]
        df_sim['snow'] = weather_matrix[:, 2]
        df_sim['wspd'] = weather_matrix[:, 3]
        
        # Scale weather
        for col in weather_cols:
            if col in wstats:
                df_sim[col] = (df_sim[col] - wstats[col]["mean"]) / wstats[col]["std"]
        
        # Scale spatial
        df_sim["grid_lat_norm"] = (df_sim["grid_lat"] - sstats["lat_mean"]) / sstats["lat_std"]
        df_sim["grid_lng_norm"] = (df_sim["grid_lng"] - sstats["lng_mean"]) / sstats["lng_std"]
        df_sim["lat2"] = df_sim["grid_lat_norm"]**2
        df_sim["lng2"] = df_sim["grid_lng_norm"]**2
        df_sim["lat_lng"] = df_sim["grid_lat_norm"] * df_sim["grid_lng_norm"]
        
        # ----------------------------------------------------------------
        # C) Sample β
        # ----------------------------------------------------------------
        X = patsy.build_design_matrices([design_info], df_sim, return_type="dataframe")[0]
        xcols = X.columns.tolist()
        
        beta_hat = res.params.loc[xcols].values
        cov_hat = res.cov_params().loc[xcols, xcols].values
        beta_s = rng.multivariate_normal(mean=beta_hat, cov=cov_hat)
        
        # ----------------------------------------------------------------
        # D) Calculate μ and Sample Crashes
        # ----------------------------------------------------------------
        df_aligned = df_sim.loc[X.index]
        E = df_aligned["exposure_min"].values
        
        eta = X.values @ beta_s
        eta = np.clip(eta, -20, 20)
        mu = E * np.exp(eta)
        mu = np.clip(mu, 0, 1e6)
        
        if use_nb and alpha_nb > 0:
            shape = 1.0 / alpha_nb
            lam = rng.gamma(shape=shape, scale=alpha_nb * mu)
            lam = np.clip(lam, 0, 1e6)
            y = rng.poisson(lam)
        else:
            y = rng.poisson(mu)
        
        totals[s] = int(y.sum())
    
    return totals

# ============================================================================
# STEP 4: Run Simulations
# ============================================================================
print("\n" + "="*70)
print("RUNNING YEAR-LEVEL WEATHER BOOTSTRAP (day-of-year mapping)")
print("="*70)

print("\nModel: Poisson")
tot_p = simulate_totals_year_weather(poisson_res, use_nb=False)

print("\nModel: Negative Binomial")
tot_n = simulate_totals_year_weather(nb_res, use_nb=True)

def summarize(arr):
    q05, q50, q95 = np.quantile(arr, [0.05, 0.5, 0.95])
    return float(q05), float(q50), float(q95)

p05, p50, p95 = summarize(tot_p)
n05, n50, n95 = summarize(tot_n)

print("\n" + "="*70)
print("RESULTS WITH YEAR-LEVEL WEATHER BOOTSTRAP")
print("="*70)
print(f"Poisson: 5th = {p05:.0f}, Median = {p50:.0f}, 95th = {p95:.0f}")
print(f"NegBin:  5th = {n05:.0f}, Median = {n50:.0f}, 95th = {n95:.0f}")
print(f"\nPoisson 90% CI: [{p05:.0f}, {p95:.0f}], width = {p95-p05:.0f} ({(p95-p05)/p50*100:.1f}%)")
print(f"NegBin  90% CI: [{n05:.0f}, {n95:.0f}], width = {n95-n05:.0f} ({(n95-n05)/n50*100:.1f}%)")


# ---- Save ----
mc_df = pd.DataFrame({
    "model": ["poisson"]*S + ["neg_bin"]*S,
    "total_2025": np.concatenate([tot_p, tot_n])
})
mc_path = OUT_DIR / "risk_mc_2025_totals_bike_all.parquet"
mc_df.to_parquet(mc_path, index=False)
print("\nSaved:", mc_path)

def save_model_artifacts(res, model_name: str):
    params_path = OUT_DIR / f"{model_name}_params.parquet"
    cov_path    = OUT_DIR / f"{model_name}_cov.parquet"
    res.params.to_frame("coef").reset_index(names="term").to_parquet(params_path, index=False)
    res.cov_params().reset_index().to_parquet(cov_path, index=False)
    return params_path, cov_path

p_params, p_cov = save_model_artifacts(poisson_res, "poisson_bike_all")
n_params, n_cov = save_model_artifacts(nb_res, "negbin_bike_all")

print("Saved:", p_params, p_cov)
print("Saved:", n_params, n_cov)

cmp = pd.DataFrame([
    {"model":"poisson", "aic": float(poisson_res.aic), "dispersion": float(np.sum(poisson_res.resid_pearson**2)/poisson_res.df_resid),
     "q50_total_2025": p50, "q05_total_2025": p05, "q95_total_2025": p95},
    {"model":"neg_bin", "aic": float(nb_res.aic), "dispersion": float(np.sum(nb_res.resid_pearson**2)/nb_res.df_resid),
     "q50_total_2025": n50, "q05_total_2025": n05, "q95_total_2025": n95},
])
cmp_path = OUT_DIR / "model_comparison_bike_all.parquet"
cmp.to_parquet(cmp_path, index=False)
print("Saved:", cmp_path)

cmp

PREPARING YEAR-LEVEL WEATHER BOOTSTRAP (day-of-year mapping)
  Year 2020: 8734 unique (dayofyear, hour) combinations
  Year 2021: 8709 unique (dayofyear, hour) combinations
  Year 2022: 8760 unique (dayofyear, hour) combinations
  Year 2023: 8760 unique (dayofyear, hour) combinations
  Year 2024: 8784 unique (dayofyear, hour) combinations

Day 15, hour 14 across years:
  2020: temp=+5.6°C, prcp=0.0mm
  2021: temp=+0.9°C, prcp=0.0mm
  2022: temp=-10.6°C, prcp=0.0mm
  2023: temp=-1.6°C, prcp=0.0mm
  2024: temp=-5.3°C, prcp=0.0mm

Prepared 2025 data: 412528 rows

RUNNING YEAR-LEVEL WEATHER BOOTSTRAP (day-of-year mapping)

Model: Poisson
  Simulation 0/500...
  Simulation 50/500...
  Simulation 100/500...
  Simulation 150/500...
  Simulation 200/500...
  Simulation 250/500...
  Simulation 300/500...
  Simulation 350/500...
  Simulation 400/500...
  Simulation 450/500...

Model: Negative Binomial
  Simulation 0/500...
  Simulation 50/500...
  Simulation 100/500...
  Simulation 150/500...
  

,model,aic,dispersion,q50_total_2025,q05_total_2025,q95_total_2025
0,poisson,240602.006487,1.451729,6575.0,6418.95,6745.05
1,neg_bin,250526.526426,1.041182,7350.0,7113.95,7606.30


In [22]:
# Observed crashes 2025 (citywide)
obs = con.execute(f"""
SELECT
  date_trunc('month', hour_ts) AS month_ts,
  SUM(y_bike) AS y_obs
FROM read_parquet('{crashes_citywide_path.as_posix()}')
WHERE hour_ts >= TIMESTAMP '2025-01-01'
  AND hour_ts <  TIMESTAMP '2026-01-01'
GROUP BY 1
ORDER BY 1;
""").fetch_df()

# ============================================================================
# Prepare df2025 with ALL features for expected_monthly
# ============================================================================
# df2025 already loaded in previous cell, but needs all derived features

# Apply weather scaling
for col in weather_cols:
    if col in wstats:
        df2025[col] = (df2025[col] - wstats[col]["mean"]) / wstats[col]["std"]

# Apply spatial scaling
df2025["grid_lat_norm"] = (df2025["grid_lat"] - sstats["lat_mean"]) / sstats["lat_std"]
df2025["grid_lng_norm"] = (df2025["grid_lng"] - sstats["lng_mean"]) / sstats["lng_std"]

# Spatial features
df2025["lat2"] = df2025["grid_lat_norm"]**2
df2025["lng2"] = df2025["grid_lng_norm"]**2
df2025["lat_lng"] = df2025["grid_lat_norm"] * df2025["grid_lng_norm"]

print("df2025 prepared with all features")

# ============================================================================
# Expected from grid-model: sum over cells of mu_cellhour
# ============================================================================
def expected_monthly(res, label: str):
    import patsy
    design_info = res.model.data.design_info
    X = patsy.build_design_matrices([design_info], df2025, return_type="dataframe")[0]
    
    # Align to X's index
    df_aligned = df2025.loc[X.index]
    
    beta = res.params.loc[X.columns].values
    eta = X.values @ beta
    
    # Clip for safety
    eta = np.clip(eta, -20, 20)
    
    mu = df_aligned["exposure_min"].values * np.exp(eta)
    
    tmp = pd.DataFrame({"hour_ts": df_aligned["hour_ts"].values, "mu": mu})
    tmp["month_ts"] = pd.to_datetime(tmp["hour_ts"]).dt.to_period('M').dt.to_timestamp()
    out = tmp.groupby("month_ts", as_index=False)["mu"].sum()
    out = out.rename(columns={"mu": f"y_pred_{label}"})
    return out

pred_p = expected_monthly(poisson_res, "poisson")
pred_n = expected_monthly(nb_res, "negbin")

# Merge predictions (wide format first)
eval_wide = obs.merge(pred_p, on="month_ts", how="left").merge(pred_n, on="month_ts", how="left")

# Convert to LONG format for dashboard
eval_long_poisson = eval_wide[["month_ts", "y_obs", "y_pred_poisson"]].copy()
eval_long_poisson["model"] = "poisson"
eval_long_poisson = eval_long_poisson.rename(columns={"y_obs": "observed", "y_pred_poisson": "pred_mean"})

eval_long_nb = eval_wide[["month_ts", "y_obs", "y_pred_negbin"]].copy()
eval_long_nb["model"] = "neg_bin"
eval_long_nb = eval_long_nb.rename(columns={"y_obs": "observed", "y_pred_negbin": "pred_mean"})

# Combine
eval_df = pd.concat([eval_long_poisson, eval_long_nb], ignore_index=True)

# Save
eval_path = OUT_DIR / "risk_eval_2025_monthly_bike_all.parquet"
eval_df.to_parquet(eval_path, index=False)

print("Saved:", eval_path)
print("\nPreview:")
print(eval_df.head(10))

# Show wide format too for inspection
print("\nWide format (for reference):")
print(eval_wide)

df2025 prepared with all features
Saved: /Users/patricknussbaum/Desktop/projects/city-bike/data/processed/risk_hourly_mc/risk_eval_2025_monthly_bike_all.parquet

Preview:
    month_ts  observed   pred_mean    model
0 2025-01-01     291.0  328.618174  poisson
1 2025-02-01     302.0  317.416231  poisson
2 2025-03-01     508.0  519.185383  poisson
3 2025-04-01     546.0  544.403365  poisson
4 2025-05-01     736.0  646.118406  poisson
5 2025-06-01     683.0  663.116768  poisson
6 2025-07-01     776.0  702.470406  poisson
7 2025-08-01     758.0  755.293659  poisson
8 2025-09-01     713.0  779.961723  poisson
9 2025-10-01     688.0  702.694071  poisson

Wide format (for reference):
     month_ts  y_obs  y_pred_poisson  y_pred_negbin
0  2025-01-01  291.0      328.618174     351.736641
1  2025-02-01  302.0      317.416231     343.931734
2  2025-03-01  508.0      519.185383     577.078929
3  2025-04-01  546.0      544.403365     609.552238
4  2025-05-01  736.0      646.118406     741.045104
5  